In [1]:
# Library Import
import numpy as np 
import pandas as pd 
import sklearn as skl


# Load the training data and the test inputs
x_train = pd.read_csv('X_train.csv', index_col = 0, header=[0, 1, 2])
y_train = pd.read_csv('y_train.csv', index_col=0)
y_train = y_train.squeeze().to_numpy() # Make y_train a NumPy array
x_test = pd.read_csv('X_test.csv', index_col = 0, header=[0, 1, 2])

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
Id                                                                         
0             3.974762  4.354650  3.394523  0.033462  5.611623  2.592655   
1             0.033636 -0.139950  0.678688 -0.553824 -0.165293  0.370275   
2             0.044094  4.285359  0.977213  3.240997  0.400350  1.026224   
3            -0.425218  0.793007  0.509624 -0.967103 -1.432252 -0.900761   
4            -1.120238 -0.503659  0.303515 -0.596549 -0.716761 -0.874363   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
Id                                                  ...                       
0           3.041094  2.815378  3.954026  2.365586  ...  0.052970  0.013487   
1          -0.314710 -0.368706 -0.437181 -0.441662  ...  0.093105  0.022865   
2           0.772464  0.807625  1.942534  1.938970  ...  0.081955  0.017371   
3          -0.501279 -0.855886 -0.556825  7.404243  ...  0.098877  0.019089   
4          -0.708101 -0.642351 -0.327327 -0.342220  ...  0.102545  0.023823   

feature                     zcr                                          \
statistics             kurtosis       max      mean    median       min   
number            06         01        01        01        01        01   
Id                                                                        
0           0.022031  70.844788  0.671387  0.035129  0.026367  0.010254   
1           0.028800  18.427612  0.538574  0.055975  0.039062  0.000000   
2           0.016673  38.401405  0.405762  0.030685  0.028809  0.002930   
3           0.016183   4.129582  0.252441  0.037955  0.024902  0.000000   
4           0.025166  16.758356  0.380371  0.027851  0.019043  0.000000   

feature                         
statistics      skew       std  
number            01        01  
Id                              
0           8.394708  0.067026  
1           3.638194  0.053879  
2           2.620369  0.016835  
3           1.898847  0.034382  
4           3.244483  0.027128  

[5 rows x 518 columns]